In [1]:
from kafka import KafkaConsumer
import time
import json
import queue
from IPython.display import clear_output

In [2]:
# This script receives messages from a Kafka topic
from kafka import KafkaConsumer

consumer = KafkaConsumer(
    "accidentes",
    bootstrap_servers="kafka-194e1e33-jose-30ef.aivencloud.com:26426",
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    client_id="demo-client",
    group_id="demo-group",
    security_protocol="SSL",
    ssl_cafile="./cert/ca.pem",
    ssl_certfile="./cert/service.cert",
    ssl_keyfile="./cert/service.key"
)



In [10]:
"""
Presenta en pantalla los últimso 20 accidentes que se han producido
para que sean monitorizados
"""
NUM_ALERTS = 10
alerts=[None for i in range(NUM_ALERTS)] #inicializar las alertas
current = 0
while True:
    raw_msgs = consumer.poll(timeout_ms=1000, max_records=1)
    #print ('Poll ralizado, mensajes {}'.format(len(raw_msgs.items())))
    for tp, msgs in raw_msgs.items():
        #Limpia la salida de la celda para evitar el scroll
        #print ('cargar {} mensajes'.format(len(msgs)))
        for msg in msgs:
            if alerts [NUM_ALERTS-1] == None or alerts[NUM_ALERTS-1]['acc'] != msg.value['acc']:
                #Elimina la alerta más antigua
                alerts.pop(0)
                #incluye en la cabecera
                alerts.append(msg.value)
        clear_output(wait=True)
        #Los elementos más nuevos están al final de la lista
        print ('Panel -----------------------------------------------------')
        for i in range(NUM_ALERTS-1,0,-1):
            if alerts[i] != None:
                print (alerts[i]['acc'],'->\t',alerts[i]['calle'],alerts[i]['num'])
        print ('-----------------------------------------------------------')
    time.sleep(1)


Panel -----------------------------------------------------
2018/34 ->	 AVENIDA DEL MEDITERRANEO NUM 32
2018/23 ->	 CALLE  VIA CARPETANA - CALLE DE CASIMIRO ESCUDERO 0
2018/43 ->	 AUTOVIA  M-30 CALZADA 1 KM. 4300
2018/20 ->	 AUTOVIA  M-30 CALZADA 1 KM. 4700
2018/76 ->	 CALLE DE ALCALA NUM 44
2018/25 ->	 CALLE DE ANCORA - PASEO DE LAS DELICIAS 0
2018/8 ->	 CALLE DE LA FUENTE CARRANTONA - CALLE DE LUIS DE HOYOS SAINZ 0
2018/16 ->	 CALLE DEL CABO LOPEZ MARTINEZ NUM 32
-----------------------------------------------------------


KeyboardInterrupt: 